In [1]:
#import statements
from pyspark.sql import SparkSession
from pyspark.sql import HiveContext
from pyspark.sql.functions import *
from pyspark.sql.functions import col, udf
from pyspark.sql.types import *
#
from datetime import datetime
import seaborn as sns
#Window
from pyspark.sql.window import Window
#misc
import functools as fc
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
!gsutil ls gs://bigdataplatform-order/War&Others/

/bin/sh: Others/: No such file or directory


In [3]:
spark = SparkSession.builder.appName('Eve_online').getOrCreate()

In [4]:
sc =spark.sparkContext

In [5]:
df_representative = pd.read_csv('gs://bigdataplatform-order/representative.csv',index_col=0)

In [6]:
df_representative.head()

,region_id,diversity,total_value,total_volume,total_num_orders
0,10000002,7933.857008,6.363859e+12,1.507947e+10,371141.017461
1,10000043,4883.157075,8.181978e+11,2.460910e+09,78487.900000
2,10000060,2327.175943,3.713254e+11,1.944459e+09,22208.451887
3,10000032,3650.008962,3.187504e+11,1.053530e+09,47678.623585
4,10000030,2706.286321,1.410632e+11,5.174368e+08,22724.729245


In [7]:
region_id_of_interest = list(df_representative[:18].region_id)

# Join Table Plan 
1. Identify the regions that are most popular
2. Get average within the subset 
3. Join the tables and store as a csv 

In [13]:
df_market =  spark.read.csv("gs://bigdataplatform-order/market-history/market_history.csv",
                            header=True, inferSchema=True)

In [14]:
df_war_items  = spark.read.csv("gs://bigdataplatform-order/War&Others/ITEMS_Agg_All.csv",
                            header=True, inferSchema=True)

In [15]:
df_WAR = spark.read.csv("gs://bigdataplatform-order/War&Others/WARS.csv",
                            header=True, inferSchema=True)

In [16]:
df_market.printSchema()

root
 |-- date: string (nullable = true)
 |-- region_id: integer (nullable = true)
 |-- type_id: integer (nullable = true)
 |-- average: double (nullable = true)
 |-- lowest: double (nullable = true)
 |-- highest: double (nullable = true)
 |-- volume: long (nullable = true)
 |-- order_count: integer (nullable = true)



In [11]:
df_war_items.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- item_type_id: integer (nullable = true)
 |-- quantity_destroyed: double (nullable = true)
 |-- singleton: integer (nullable = true)
 |-- killmail_id: double (nullable = true)
 |-- victim.character_id: double (nullable = true)
 |-- quantity_dropped: double (nullable = true)
 |-- flag: double (nullable = true)



In [12]:
df_market.select('*').show(5)

+----------+---------+-------+-------+------+-------+--------+-----------+
|      date|region_id|type_id|average|lowest|highest|  volume|order_count|
+----------+---------+-------+-------+------+-------+--------+-----------+
|2020-01-17| 10000001|     19| 100.99|100.99| 100.99|      90|          1|
|2020-01-17| 10000001|     20| 261.97|261.97| 261.97|    8856|          3|
|2020-01-17| 10000001|     34|    3.5|   3.5|    3.5|13524267|         37|
|2020-01-17| 10000001|     35|   2.99|  2.99|   2.99|60551454|         28|
|2020-01-17| 10000001|     36|  16.99| 16.99|  16.99|  654278|         30|
+----------+---------+-------+-------+------+-------+--------+-----------+
only showing top 5 rows



In [ ]:
df_market_daily = df_market.withColumn('value',col('average')*col('order_count')).groupBy("date", "region_id").\
                                agg(countDistinct('type_id').alias('diversity'),
                                          sum('value').alias('total_value') ,
                                          sum('volume').alias('total_volume') ,
                                          sum('order_count').alias('total_num_orders'))

In [ ]:
df_market_summary = df_market_daily.groupBy('date').agg(mean('diversity').alias('daily_avg_diversity'),
                                   mean('total_value').alias('daily_avg_total_value'),
                                   mean('total_num_orders').alias('daily_avg_total_num_orders'))

In [ ]:
df_market_summary.toPandas().to_csv('df_market_summary.csv')

In [15]:
#Create temporay table in SQL
df_market.createOrReplaceTempView("market")

In [20]:
#Show daily volume 
SQL = """
SELECT region_id, mean(volume) as avg_daily_vol
FROM market 
GROUP BY region_id
ORDER BY avg_daily_vol DESC
"""
spark.sql(SQL).show()

+---------+------------------+
|region_id|     avg_daily_vol|
+---------+------------------+
| 10000002|1900647.4646837946|
| 11000003|1644966.6557377048|
| 10000060| 835544.3105886092|
| 11000013| 813223.8333333334|
| 10000043|503958.79950306664|
| 10000059| 400855.8689385633|
| 10000032|288637.61168045725|
| 10000022|287754.89362446807|
| 10000049|263668.36317368597|
| 10000005|247726.55395336068|
| 10000066|235484.71921048782|
| 11000029|229294.91681574238|
| 10000063|227040.90226904565|
| 10000016|222860.60284971216|
| 11000002| 221124.6462264151|
| 10000001| 220591.3711026036|
| 10000013|219367.37182283966|
| 10000055|215026.64225960875|
| 10000050|213519.13382295595|
| 10000020|209550.80069105685|
+---------+------------------+
only showing top 20 rows



In [28]:
#Show daily diversity
SQL = """
SELECT region_id, mean(diversity) as avg_diversity
FROM 

(SELECT date, region_id, count(DISTINCT type_id ) as diversity
FROM market 
GROUP BY region_id, date
) as t1

GROUP BY region_id 
ORDER BY avg_diversity DESC 
"""
spark.sql(SQL).show(5)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1211, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while receiving
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:43665)
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 33

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:43665)
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3397, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_4830/570801968.py", line 14, in <cell line: 14>
    spark.sql(SQL).show(5)
  File "/usr/lib/spark/python/pyspark/sql/session.py", line 723, in sql
    return DataFrame(self._jsparkSession.sql(sqlQuery), self._wrapped)
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "/usr/lib/spark/python/pyspark/sql/utils.py", line 111, in deco
    return f(*a, **kw)
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/protocol.py", line 334, in get_return_value
    raise Py4JError(
py4j.protocol.Py4JError: An error occurred while calling o53.sql

During handling of the

Py4JError: An error occurred while calling o53.sql

In [33]:
market_representative = df_market.filter(col('region_id').isin(region_id_of_interest))

In [34]:
market_representative_final = market_representative.groupBy('date','type_id').agg(
    mean('average').alias('mean_average'),
    mean('lowest').alias('mean_lowest'),
    mean('highest').alias('mean_highest'),
    mean('volume').alias('mean_volume'),
    mean('order_count').alias('mean_order_count')
)

In [15]:
market_representative_final.show(5)

+----------+-------+-------------------+------------------+------------------+------------------+------------------+
|      date|type_id|       mean_average|       mean_lowest|      mean_highest|       mean_volume|  mean_order_count|
+----------+-------+-------------------+------------------+------------------+------------------+------------------+
|2020-10-13|  12787|  295.2923076923077| 295.2923076923077|             295.3|58893.307692307695|               7.0|
|2020-10-13|  18985|      8340470.83375|         7374200.0|         8995762.5|             21.25|             10.25|
|2020-10-13|  20505|6.685487179499999E8|           6.536E8|           6.732E8|               5.0|               5.0|
|2020-10-13|  21594| 30.674285714285713|30.674285714285713|30.674285714285713|            3736.0|19.571428571428573|
|2020-10-13|  22551|          3.00714E7|             3.0E7|          3.0357E7|               3.2|               3.2|
+----------+-------+-------------------+------------------+-----

In [16]:
#market_representative_final.write.csv('market_representative_final.csv')

In [8]:
#market_representative_final = pd.read_csv('market_representative_final.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'market_representative_final.csv'

In [18]:
market_representative_final.printSchema()

root
 |-- date: string (nullable = true)
 |-- type_id: integer (nullable = true)
 |-- mean_average: double (nullable = true)
 |-- mean_lowest: double (nullable = true)
 |-- mean_highest: double (nullable = true)
 |-- mean_volume: double (nullable = true)
 |-- mean_order_count: double (nullable = true)



In [17]:
df_war_items.show(5)

+---+------------+------------------+---------+-----------+-------------------+----------------+----+
|_c0|item_type_id|quantity_destroyed|singleton|killmail_id|victim.character_id|quantity_dropped|flag|
+---+------------+------------------+---------+-----------+-------------------+----------------+----+
|  0|       33890|               1.0|        0|5.1068899E7|        9.3247997E7|            null|null|
|  1|        3170|               1.0|        0|5.1068899E7|        9.3247997E7|            null|null|
|  2|       33076|              null|        0|5.1068899E7|        9.3247997E7|             1.0|null|
|  3|        5973|               1.0|        0|5.1068899E7|        9.3247997E7|            null|null|
|  4|        5445|               1.0|        0|5.1068899E7|        9.3247997E7|            null|null|
+---+------------+------------------+---------+-----------+-------------------+----------------+----+
only showing top 5 rows



In [18]:
df_WAR.show(5)

+-----------+-------------------+---------------+------+--------------------+-------------------+---------------------+-------------------+--------------------+--------------------+--------------------+-------------------+------------------+-------------------+
|killmail_id|      killmail_time|solar_system_id|war_id|       killmail_hash| http_last_modified|victim.corporation_id|victim.damage_taken|   victim.position.x|   victim.position.y|   victim.position.z|victim.ship_type_id|victim.alliance_id|victim.character_id|
+-----------+-------------------+---------------+------+--------------------+-------------------+---------------------+-------------------+--------------------+--------------------+--------------------+-------------------+------------------+-------------------+
|5.1068899E7|2016-01-01 00:46:16|    3.0002795E7|291410|d0e192095dc375dd4...|2021-07-18 21:10:01|        1.741770561E9|             3045.0|-9.103785621616903E9|-1.06768098912816...|5.476665422361162E10|            

In [20]:
df_war_items.show(5)

+---+------------+------------------+---------+-----------+-------------------+----------------+----+
|_c0|item_type_id|quantity_destroyed|singleton|killmail_id|victim.character_id|quantity_dropped|flag|
+---+------------+------------------+---------+-----------+-------------------+----------------+----+
|  0|       33890|               1.0|        0|5.1068899E7|        9.3247997E7|            null|null|
|  1|        3170|               1.0|        0|5.1068899E7|        9.3247997E7|            null|null|
|  2|       33076|              null|        0|5.1068899E7|        9.3247997E7|             1.0|null|
|  3|        5973|               1.0|        0|5.1068899E7|        9.3247997E7|            null|null|
|  4|        5445|               1.0|        0|5.1068899E7|        9.3247997E7|            null|null|
+---+------------+------------------+---------+-----------+-------------------+----------------+----+
only showing top 5 rows



In [30]:
df_item_agg = df_WAR.withColumn('date',to_date('killmail_time')).select('date','killmail_id').\
    join(df_war_items,on='killmail_id').groupBy('date','item_type_id').\
        agg(
    sum('quantity_destroyed').alias('quantity_destroyed_total'),
    sum('quantity_dropped').alias('quantity_dropped_total')
)

In [31]:
df_item_agg.show(5)

+----------+------------+------------------------+----------------------+
|      date|item_type_id|quantity_destroyed_total|quantity_dropped_total|
+----------+------------+------------------------+----------------------+
|2016-01-03|         519|                    23.0|                  30.0|
|2017-05-03|       41220|                     2.0|                  null|
|2016-01-12|        2889|                    29.0|                  16.0|
|2017-05-20|       15999|                    null|                   1.0|
|2016-01-01|       10246|                    16.0|                  16.0|
+----------+------------+------------------------+----------------------+
only showing top 5 rows



In [36]:
df_item_agg_market = df_item_agg.join(market_representative_final,
         (market_representative_final.date == df_item_agg.date)&(market_representative_final.type_id == df_item_agg.item_type_id))

In [ ]:
df_item_agg_market.show(5)